In [ ]:
# %load calculateWeight.py

import createData

import os
import numpy as np
import gensim
from scipy.linalg import norm
from gensim.models import Word2Vec
from ckiptagger import data_utils
from ckiptagger import WS, POS, NER

# confidence standard
STANDARD = 0.5

# model for returning similar result
class Cal_Model(object):

    def __init__(self, question_num=0, ans_num=1):
        self.ques, self.ans = createData.read_excel(question_num, ans_num)
        self.ws = WS("./data")
        self.pos = POS("./data")
        self.ner = NER("./data")
        self.model = Word2Vec.load('./20180309wiki_model.bin')

    # input_question compare each quesstion return answer
    def compare_data(self, ques_input):
        compare_value = dict()
        for q in self.ques:
            compare_value[q] = self.cal_similarity(ques_input, q)
            
        # sorted by compare_value.value------> list[ tuple ]
        ques_value = sorted(compare_value.items(),key=lambda item:item[1],reverse=True)
        
        ques_ans = dict()
        for q, a in zip(self.ques, self.ans):
            ques_ans[q] = a

        result = []
        for item in ques_value:
            if item[1] > STANDARD:
                result.append(ques_ans[item[0]])
            else:
                break
        
        return result

    def cal_similarity(self, s1, s2):
        v1, v2 = self.cal_vector(s1), self.cal_vector(s2)
        return np.dot(v1, v2) / (norm(v1) * norm(v2))

    def cal_vector(self, sentence):
        words = self.ws([sentence])
        f = open('stopWords.txt',encoding='utf-8')
        for line in f:
            if line in words[0]:
                words[0].remove(line)
        f.close()

        v = np.zeros(250)
        for word in words[0]:
            if word in self.model.wv.vocab:
                v += self.model[word]
            else:
                word= np.zeros((self.model.vector_size), dtype=np.float32)
        v /= len(words)

        return v

if __name__== '__main__':

    # 感覺(question_num=0)--->歌名
    #m_feeling = Cal_Model(question_num=0)
    #r_feeling = m_feeling.compare_data("感覺")

    # 歌詞(question_num=3)--->歌名
    m_lyrcs = Cal_Model(question_num=3)
    r_lyrcs = m_lyrcs.compare_data("歌詞")
    for row in r_lyrcs:
        print(row)

    # 歌名(question_num=1)--->歌名
    r_title = Cal_Model(question_num=1).compare_data("歌名")
